In [1]:
# Import basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
from pathlib import Path
import os


In [2]:
# Define folder paths (using Path for cross-platform compatibility)
models_folder = '../models'
plots_folder = '../plots'
temp_folder = '../temp'
data_folder = '../data'
logs_folder = '../logs'
sample_file = 'sample_clean_a_agile_only.xlsx'
data_file = 'ISBSG2016R1_1_Formatted4CSVAgileOnly'

In [3]:
# Sets up an automatic timestamp printout after each Jupyter cell execution 
# and configures the default visualization style.
from IPython import get_ipython

def setup_timestamp_callback():
    """Setup a timestamp callback for Jupyter cells without clearing existing callbacks."""
    ip = get_ipython()
    if ip is not None:
        # Define timestamp function
        def print_timestamp(*args, **kwargs):
            """Print timestamp after cell execution."""
            print(f"Cell executed at: {datetime.now()}")
        
        # Check if our callback is already registered
        callbacks = ip.events.callbacks.get('post_run_cell', [])
        for cb in callbacks:
            if hasattr(cb, '__name__') and cb.__name__ == 'print_timestamp':
                # Already registered
                return
                
        # Register new callback if not already present
        ip.events.register('post_run_cell', print_timestamp)
        print("Timestamp printing activated.")
    else:
        print("Not running in IPython/Jupyter environment.")

# Setup timestamp callback
setup_timestamp_callback()

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

Timestamp printing activated.
Cell executed at: 2025-05-29 18:08:18.794218


In [4]:
# Load the data
print("Loading data...")

file_path = f"{data_folder}/{sample_file}"  # should use data_file for model training
file_name_no_ext = Path(file_path).stem                # 'ISBSG2016R1.1 - FormattedForCSV'
print(file_name_no_ext)


df = pd.read_excel(file_path)

Loading data...
sample_clean_a_agile_only
Cell executed at: 2025-05-29 18:08:19.175501


In [5]:
# Cleans and standardizes string columns and column names by removing spaces, 
# converting to lowercase, and normalizing formatting.


"""

What it does:
Handles null values: If the value is missing (null/NaN), it returns it as is.
Converts to string, trims, lowers: Ensures it's a string, removes leading/trailing spaces, converts to lowercase.
Reduces whitespace: Replaces multiple spaces/tabs with a single space.
Strips punctuation: Removes any trailing semicolons, commas, or periods.
Removes parentheses: Gets rid of all ( and ) characters.
Cleans up semicolon formatting: Ensures no repeated or weirdly spaced semicolons.

Goal:
To make category strings (like " Dog;Cat ; Fish ") consistent and simple (e.g., "dog;cat; fish").

"""

# Cleaning category values
def clean_category(val):
    if pd.isnull(val):
        return val
    val = str(val).strip().lower()
    val = re.sub(r'\s+', ' ', val)
    val = val.rstrip(';,.')
    val = val.replace('(', '').replace(')', '')
    val = re.sub(r';\s*;', ';', val)
    val = re.sub(r';\s+', '; ', val)
    return val

"""
What it does:
Trims spaces: Removes leading/trailing spaces in column names.
Lowercases everything: For consistency.
Replaces spaces and dashes with underscores: Makes column names more code-friendly.
Removes double underscores: Collapses them to single underscores.
Removes parentheses: Deletes all ( and ) from column names.
Replaces special characters: Turns < into less_than_, > into great_than_, and removes ?.

Goal:
To standardize column names, so they're easier to work with in code (e.g., "Age (years)?" → "age_years").

"""

# Clean column names
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(' ', '_', regex=False)
      .str.replace('-', '_', regex=False)
      .str.replace('__', '_', regex=False)
      .str.replace('(', '', regex=False)
      .str.replace(')', '', regex=False)
      .str.replace('<', 'less_than_', regex=False)
      .str.replace('>', 'great_than_', regex=False)
      .str.replace('?', '', regex=False)
)

# After cleaning columns, re-detect categorical columns
cat_cols = df.select_dtypes(include=['object', 'category']).columns
for col in cat_cols:
    df[col] = df[col].map(clean_category)


Cell executed at: 2025-05-29 18:08:19.194139


In [6]:
# Find columns with semicolons
def find_semicolon_columns(df, min_count=1, sample_size=1000):
    """
    Returns a list of columns in df where at least `min_count` cell(s) 
    contain a semicolon (';'). To improve speed, only the first `sample_size` 
    non-null values in each column are scanned by default.
    """
    semicolon_cols = []
    for col in df.columns:
        # Drop missing values, convert to string, sample up to `sample_size`
        sample = df[col].dropna().astype(str).head(sample_size)
        count = sample.str.contains(';').sum()
        if count >= min_count:
            semicolon_cols.append(col)
    return semicolon_cols

# Usage:
cols_with_semicolons = find_semicolon_columns(df)
print("Columns with semicolon-separated values:", cols_with_semicolons)


Columns with semicolon-separated values: ['external_eef_organisation_type', 'project_prf_application_type', 'process_pmf_development_methodologies', 'tech_tf_client_roles', 'tech_tf_server_roles']
Cell executed at: 2025-05-29 18:08:19.232564


In [7]:
print("Current columns:", df.columns.tolist())

Current columns: ['isbsg_project_id', 'external_eef_data_quality_rating', 'project_prf_year_of_project', 'external_eef_industry_sector', 'external_eef_organisation_type', 'project_prf_application_group', 'project_prf_application_type', 'project_prf_development_type', 'tech_tf_development_platform', 'tech_tf_language_type', 'tech_tf_primary_programming_language', 'project_prf_functional_size', 'project_prf_relative_size', 'project_prf_normalised_work_effort_level_1', 'project_prf_normalised_work_effort', 'project_prf_normalised_level_1_pdr_ufp', 'project_prf_normalised_pdr_ufp', 'project_prf_defect_density', 'project_prf_speed_of_delivery', 'project_prf_manpower_delivery_rate', 'project_prf_project_elapsed_time', 'project_prf_team_size_group', 'project_prf_max_team_size', 'project_prf_case_tool_used', 'process_pmf_development_methodologies', 'process_pmf_prototyping_used', 'process_pmf_docs', 'tech_tf_architecture', 'tech_tf_client_server', 'tech_tf_client_roles', 'tech_tf_server_roles'

In [8]:
# Save the entire cleaned DataFrame (not just the column names) to CSV
df.to_csv(f'../data/{file_name_no_ext}.csv', index=False)

print(f"Save the entire cleaned DataFrame (not just the column names) to CSV:  '{data_folder}/{file_name_no_ext}_all_categorical_unique_values_beforeDropping.txt'")

Cell executed at: 2025-05-29 18:08:19.256239


In [9]:
# Clean data
# Cleans, de-duplicates, and sorts semicolon-separated categorical values in specified columns.
# Clean and sort semicolon-separated categorical values
# Clean data
# Cleans, de-duplicates, and sorts semicolon-separated categorical values in specified columns.
def clean_and_sort_semicolon(val, apply_standardization=False):
    """Clean and standardise a semicolon-separated categorical string."""
    if pd.isnull(val):
        return val
    
    # Convert to string in case of mixed types
    val_str = str(val).strip()
    
    # Handle empty strings
    if not val_str or val_str.lower() == 'nan':
        return None
    
    # Split, strip, lower, remove trailing punctuation
    parts = []
    for p in val_str.split(';'):
        stripped_p = p.strip()
        if stripped_p and stripped_p.lower() != 'nan':  # Only process non-empty parts after stripping
            # Normalize internal multiple spaces to a single space
            cleaned_p = re.sub(r'\s+', ' ', stripped_p)
            cleaned_p = cleaned_p.lower().rstrip(';,.')
            
            # Apply standardization rules if requested
            if apply_standardization:
                cleaned_p = apply_individual_standardization(cleaned_p)
            
            parts.append(cleaned_p)
    
    # Remove duplicates, sort
    if parts:
        parts = sorted(set(parts))
        return '; '.join(parts)
    else:
        return None

def apply_individual_standardization(val):
    """Apply standardization rules to individual values within semicolon-separated strings."""
    if not val:
        return val
    
    # Normalize whitespace first
    val = re.sub(r'\s+', ' ', val.strip())
    
    # Apply Excel-style cleaning rules (in order)
    # Replace " ;" with ";"
    val = val.replace(' ;', ';')
    # Replace "; " with ";" 
    val = val.replace('; ', ';')
    # Replace " & " with "_"
    val = val.replace(' & ', '_')
    # Replace "&/or" with "_"
    val = val.replace('&/or', '_')
    # Replace " &" with "_"
    val = val.replace(' &', '_')
    # Replace "/" with "_"
    val = val.replace('/', '_')
    # Replace ": " with "_"
    val = val.replace(': ', '_')
    # Replace " (" with "_"
    val = val.replace(' (', '_')
    # Replace "(" with ""
    val = val.replace('(', '')
    # Replace ")" with ""
    val = val.replace(')', '')
    # Replace " + " with "_"
    val = val.replace(' + ', '_')
    
    # Clean up any double underscores or trailing underscores
    val = re.sub(r'_+', '_', val)  # Replace multiple underscores with single
    val = val.strip('_')  # Remove leading/trailing underscores
    
    # Specific standardization rules for individual components (after cleaning)
    standardization_map = {
        'stand alone': 'stand-alone',
        'stand-alone': 'stand-alone',
        'client server': 'client-server',
        'mathematically intensive': 'mathematically-intensive',
        'mathematically intensive application': 'mathematically-intensive application',
    }
    
    # Check if value matches any standardization rule
    if val in standardization_map:
        return standardization_map[val]
    
    # Remove question mark from web dev
    if val.replace('?', '').strip() == 'web':
        return 'web'
    
    # Clean up common abbreviations and inconsistencies
    val = re.sub(r'\bpsp\b', 'personal_software_process', val)
    val = re.sub(r'\bjad\b', 'joint_application_development', val)
    
    return val

# Standardizes specific categorical columns by normalizing case and correcting inconsistent formatting.
def standardize_single_value(val):
    """Standardize individual categorical values (for non-semicolon columns)."""
    if pd.isnull(val):
        return val
    
    # Convert to string and normalize
    val_str = str(val).strip().lower()
    
    # Handle empty strings or 'nan' strings
    if not val_str or val_str == 'nan':
        return None
    
    # Apply the same standardization logic
    return apply_individual_standardization(val_str)


Cell executed at: 2025-05-29 18:08:19.264686


In [10]:
# cols with semicolon as seperator is stored in cols_with_semicolons

# Manual specification for your case:
cols_semicolon_with_standardization = ['project_prf_application_group']
cols_single_standardization = ['tech_tf_architecture', 'tech_tf_web_development']

# Clean semicolon-separated columns (without standardization)
print("=== CLEANING SEMICOLON-SEPARATED COLUMNS ===")
for col in cols_with_semicolons:
    if col in df.columns:
        print(f"Cleaning column: {col}")
        original_unique = len(df[col].dropna().unique())
        df[col] = df[col].map(clean_and_sort_semicolon)
        new_unique = len(df[col].dropna().unique())
        print(f"  Unique values: {original_unique} → {new_unique}")
        
        # Show sample values
        sample_vals = df[col].dropna().unique()[:3]
        print(f"  Sample values: {[str(v) for v in sample_vals]}")
    else:
        print(f"Warning: Column '{col}' not found in dataframe")

# Clean semicolon-separated columns WITH standardization
print(f"\n=== CLEANING SEMICOLON COLUMNS WITH STANDARDIZATION ===")
for col in cols_semicolon_with_standardization:
    if col in df.columns:
        print(f"Cleaning column: {col}")
        original_unique = len(df[col].dropna().unique())
        df[col] = df[col].map(lambda x: clean_and_sort_semicolon(x, apply_standardization=True))
        new_unique = len(df[col].dropna().unique())
        print(f"  Unique values: {original_unique} → {new_unique}")
        
        # Show sample values
        sample_vals = df[col].dropna().unique()[:3]
        print(f"  Sample values: {[str(v) for v in sample_vals]}")
    else:
        print(f"Warning: Column '{col}' not found in dataframe")

# Apply standardization to single-value columns
print(f"\n=== STANDARDIZING SINGLE-VALUE COLUMNS ===")
for col in cols_single_standardization:
    if col in df.columns:
        print(f"Standardizing column: {col}")
        original_unique = len(df[col].dropna().unique())
        df[col] = df[col].map(standardize_single_value)
        new_unique = len(df[col].dropna().unique())
        print(f"  Unique values: {original_unique} → {new_unique}")
    else:
        print(f"Warning: Column '{col}' not found in dataframe")

# Special case for language type (uppercase normalization)
if 'tech_tf_language_type' in df.columns:
    print(f"\n=== NORMALIZING LANGUAGE TYPE ===")
    print("Normalizing tech_tf_language_type to uppercase")
    original_unique = len(df['tech_tf_language_type'].dropna().unique())
    df['tech_tf_language_type'] = df['tech_tf_language_type'].astype(str).str.upper().str.strip()
    # Replace 'NAN' with actual NaN
    df['tech_tf_language_type'] = df['tech_tf_language_type'].replace('NAN', pd.NA)
    new_unique = len(df['tech_tf_language_type'].dropna().unique())
    print(f"  Unique values: {original_unique} → {new_unique}")

# Verification function to check the cleaning results
def verify_semicolon_cleaning(df, columns):
    """Verify that semicolon-separated columns are properly cleaned."""
    print("\n=== VERIFICATION RESULTS ===")
    for col in columns:
        if col in df.columns:
            print(f"\nColumn: {col}")
            unique_vals = df[col].dropna().unique()
            print(f"  Total unique values: {len(unique_vals)}")
            
            # Check for unsorted or duplicate issues
            problematic = []
            for val in unique_vals:
                if ';' in str(val):
                    parts = str(val).split(';')
                    parts_stripped = [p.strip() for p in parts]
                    if parts_stripped != sorted(set(parts_stripped)):
                        problematic.append(val)
            
            if problematic:
                print(f"  ⚠️  Potentially problematic values: {problematic[:3]}")
            else:
                print("  ✓ All semicolon-separated values appear properly sorted and deduplicated")
            
            # Check for consistency in separators
            semicolon_vals = [v for v in unique_vals if ';' in str(v)]
            if semicolon_vals:
                inconsistent_separators = [v for v in semicolon_vals if '; ' not in str(v) and ';' in str(v)]
                if inconsistent_separators:
                    print(f"  ⚠️  Inconsistent separators: {inconsistent_separators[:3]}")
                else:
                    print("  ✓ All semicolon separators are consistent ('; ')")
            
            # Show sample of values
            sample_vals = [str(v) for v in unique_vals[:3] if pd.notna(v)]
            print(f"  Sample values: {sample_vals}")

# Run verification on all semicolon columns
all_semicolon_cols = cols_with_semicolons + cols_semicolon_with_standardization
verify_semicolon_cleaning(df, all_semicolon_cols)

# Show detailed before/after comparison for a few problematic values
def show_detailed_cleaning_examples(df, columns, max_examples=3):
    """Show detailed before/after examples of cleaning for verification."""
    print(f"\n=== DETAILED CLEANING EXAMPLES ===")
    
    for col in columns:
        if col not in df.columns:
            continue
            
        print(f"\nColumn: {col}")
        
        # Get some complex values (those with semicolons)
        complex_vals = [v for v in df[col].dropna().unique() if ';' in str(v)][:max_examples]
        
        if complex_vals:
            print("  Complex semicolon-separated values found:")
            for i, val in enumerate(complex_vals, 1):
                print(f"    {i}. '{val}'")
                # Show the individual parts
                parts = str(val).split(';')
                for j, part in enumerate(parts):
                    print(f"       Part {j+1}: '{part.strip()}'")
        else:
            # Show simple values
            simple_vals = df[col].dropna().unique()[:max_examples]
            print("  Sample values (no semicolons):")
            for i, val in enumerate(simple_vals, 1):
                print(f"    {i}. '{val}'")

show_detailed_cleaning_examples(df, all_semicolon_cols)

# Function to identify potential remaining issues
def identify_remaining_issues(df, columns):
    """Identify potential issues that might still need attention."""
    print(f"\n=== POTENTIAL REMAINING ISSUES ===")
    
    issues_found = False
    
    for col in columns:
        if col not in df.columns:
            continue
            
        col_issues = []
        unique_vals = df[col].dropna().unique()
        
        for val in unique_vals:
            val_str = str(val)
            
            # Check for various potential issues that should be cleaned
            if ' & ' in val_str:
                col_issues.append(f"Still contains ' & ': '{val_str}'")
            if '&/or' in val_str:
                col_issues.append(f"Still contains '&/or': '{val_str}'")
            if ' &' in val_str:
                col_issues.append(f"Still contains ' &': '{val_str}'")
            if '/' in val_str:
                col_issues.append(f"Still contains '/': '{val_str}'")
            if ': ' in val_str:
                col_issues.append(f"Still contains ': ': '{val_str}'")
            if ' (' in val_str or '(' in val_str or ')' in val_str:
                col_issues.append(f"Still contains parentheses: '{val_str}'")
            if ' + ' in val_str:
                col_issues.append(f"Still contains ' + ': '{val_str}'")
            if '  ' in val_str:  # Double spaces
                col_issues.append(f"Contains double spaces: '{val_str}'")
            if val_str.endswith(' ') or val_str.startswith(' '):
                col_issues.append(f"Has leading/trailing spaces: '{val_str}'")
            if re.search(r'[A-Z]', val_str):  # Contains uppercase
                col_issues.append(f"Contains uppercase: '{val_str}'")
            if '__' in val_str:  # Double underscores
                col_issues.append(f"Contains double underscores: '{val_str}'")
        
        if col_issues:
            print(f"\nColumn: {col}")
            issues_found = True
            for issue in col_issues[:5]:  # Show first 5 issues
                print(f"  - {issue}")
            if len(col_issues) > 5:
                print(f"  ... and {len(col_issues) - 5} more issues")
    
    if not issues_found:
        print("✓ No obvious formatting issues detected!")

# Show examples of transformations
def show_transformation_examples():
    """Show examples of how the cleaning rules transform values."""
    print(f"\n=== TRANSFORMATION EXAMPLES ===")
    
    test_cases = [
        "workflow support & management",
        "data entry &/or validation", 
        "file &/or print server",
        "html/web server: security",
        "client (desktop) application",
        "database + file server",
        "agile development ; scrum",
        "web development: html & css"
    ]
    
    print("Original → Cleaned:")
    for case in test_cases:
        cleaned = apply_individual_standardization(case.lower())
        print(f"  '{case}' → '{cleaned}'")

show_transformation_examples()

identify_remaining_issues(df, all_semicolon_cols)

# Additional verification: Check for common issues
print(f"\n=== ADDITIONAL CHECKS ===")
for col in all_semicolon_cols:
    if col in df.columns:
        # Check for trailing/leading spaces in parts
        problem_vals = []
        for val in df[col].dropna().unique():
            if ';' in str(val):
                parts = str(val).split(';')
                for part in parts:
                    if part != part.strip():
                        problem_vals.append(val)
                        break
        
        if problem_vals:
            print(f"Column {col}: Found values with untrimmed parts: {problem_vals[:2]}")
        else:
            print(f"Column {col}: ✓ All parts properly trimmed")

=== CLEANING SEMICOLON-SEPARATED COLUMNS ===
Cleaning column: external_eef_organisation_type
  Unique values: 23 → 23
  Sample values: ['banking; communications; education institution; government; medical and health care; transport & storage; wholesale & retail trade', 'government', 'community services']
Cleaning column: project_prf_application_type
  Unique values: 31 → 31
  Sample values: ['surveillance and security', 'business application', 'complex process control; workflow support & management']
Cleaning column: process_pmf_development_methodologies
  Unique values: 8 → 8
  Sample values: ['agile development', 'agile development; unified process', 'agile development; personal software process psp; unified process']
Cleaning column: tech_tf_client_roles
  Unique values: 10 → 10
  Sample values: ['data entry & validation; data retrieval & presentation; web/html browser', 'web public interface', 'data entry & validation; data retrieval & presentation; run a computer-human interface; 

In [11]:
# Writes the unique values of all categorical columns to a text file for reference or auditing.

cat_cols = df.select_dtypes(include=['object', 'category']).columns

with open(f"{temp_folder}/{file_name_no_ext}_all_categorical_unique_values_beforeDropping.txt", 'w') as f:
    for col in cat_cols:
        f.write(f"Column: {col} (n_unique = {df[col].nunique()})\n")
        f.write(f"{df[col].unique()}\n")
        f.write('-' * 40 + '\n')

print(f"Before Dropping: Unique values for categorical columns saved to '{temp_folder}/{file_name_no_ext}_all_categorical_unique_values_beforeDropping.txt'")

Before Dropping: Unique values for categorical columns saved to '../temp/sample_clean_a_agile_only_all_categorical_unique_values_beforeDropping.txt'
Cell executed at: 2025-05-29 18:08:19.348833


In [12]:
# Save the entire cleaned DataFrame (not just the column names) to CSV
df.to_csv(f"{data_folder}/{file_name_no_ext}_cleaned.csv", index=False)

Cell executed at: 2025-05-29 18:08:19.365831
